##### Imports:

In [1]:
from utils import separate_for_training, preprocess, calculate_emission_from_chroma, calculate_mu_from_chroma, calculate_transition_probabilites
import pickle
from chroma import get_chromagram
import pandas as pd
from tqdm import tqdm
import numpy as np

##### Steps:

1. Training / Testing Data Split
2. Create Chromagram from Training Data
3. Create HMM Initialization Components
    - Initial State Probabilities
    - Transition Probability Matrix
    - Mu Value
    - Emission Matrix
4. Create HMM Object
5. Fit / Train HMM

##### Training / Test Data Split:

In [2]:
# Load data and split into training and test
piece_name_dict = preprocess('dataset.pkl')

training_piece_names, _, test_piece_names = separate_for_training(piece_name_dict, 0.8, 0.)

with open(r"dataset.pkl", 'rb') as data:
    midi_data = pickle.load(data)

100%|██████████| 5762/5762 [00:02<00:00, 2050.53it/s]


##### Create Chromagram from Training Data:

In [3]:
song_chromagrams = []
for song_name in tqdm(list(training_piece_names)):
    indiv_chroma = get_chromagram(song_name, midi_data)
    song_chromagrams.append(indiv_chroma)

chromagram = pd.concat(song_chromagrams)
chromagram.head(6)

100%|██████████| 4609/4609 [00:24<00:00, 184.39it/s]


,C,C#,D,D#,E,F,F#,G,G#,A,A#,B,Chord Actual
0,0,316,0,0,0,0,0,0,147,0,0,0,Em
1,0,316,0,0,0,0,0,0,147,0,0,0,Em
2,0,316,0,0,0,0,0,0,147,0,0,0,Em
3,0,316,0,0,0,0,0,0,147,0,0,0,Em
4,0,316,0,0,0,0,0,0,147,0,0,0,Em
5,0,316,0,0,0,0,0,0,147,0,0,0,Em


##### Create HMM Components:

###### Initial State Probabilities:

In [4]:
def calculate_init_prob(song_chromagrams):
    first_chords = []
    for song_chroma in tqdm(song_chromagrams):
        first_chord = song_chroma['Chord Actual'].values[0]
        first_chords.append(first_chord)
    
    first_chord_counts = np.unique(first_chords, return_counts=True)

    return pd.Series(first_chord_counts[1]/first_chord_counts[1].sum(), index=first_chord_counts[0])

In [5]:
initial_state_probabilties = calculate_init_prob(song_chromagrams)
initial_state_probabilties

100%|██████████| 4609/4609 [00:00<00:00, 37510.62it/s]


A        0.020395
A#       0.000217
B        0.120634
Bm       0.155782
C        0.000217
C#dim    0.000217
C#m      0.062486
D        0.117162
D#m      0.020395
E        0.116511
Em       0.060100
F#       0.021914
F#m      0.020395
G        0.118898
G#m      0.164678
dtype: float64

###### Transition Matrix:

In [6]:
transition_prob_matrix = calculate_transition_probabilites(chromagram)
transition_prob_matrix.head()

following_chords,A,A#,A#dim,B,Bm,C,C#dim,C#m,D,D#,...,Em,F,F#,F#m,G,G#,G#m,dim,<E>,<S>
initial_chords,,,,,,,,,,,,,,,,,,,,,
A,0.861214,0.000,0.000073,0.006487,0.043589,0.000073,0.002989,0.002916,0.019681,0.00000,...,0.009840,0.0,0.001166,0.011152,0.020920,0.000073,0.008747,0.000073,0.0,0
A#,0.000000,0.875,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.09375,...,0.031250,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0
A#dim,0.005391,0.000,0.878706,0.005391,0.016173,0.000000,0.000000,0.010782,0.010782,0.00000,...,0.008086,0.0,0.005391,0.000000,0.013477,0.000000,0.032345,0.000000,0.0,0
B,0.000908,0.000,0.000000,0.862980,0.005448,0.000091,0.000091,0.013620,0.008989,0.00000,...,0.001907,0.0,0.048851,0.000636,0.003269,0.000000,0.027422,0.000000,0.0,0
Bm,0.044079,0.000,0.000000,0.001527,0.863250,0.000133,0.000000,0.000664,0.022969,0.00000,...,0.012215,0.0,0.000398,0.022504,0.026753,0.000000,0.003651,0.000000,0.0,0


###### Mu Value:

In [7]:
mu = calculate_mu_from_chroma(chromagram)
mu

C     11.341131
C#    37.151099
D     24.881187
D#    34.657234
E     29.062937
F     16.501733
F#    45.153889
G     16.587213
G#    40.491961
A     21.787228
A#    25.193985
B     49.429562
dtype: float64

###### Emission Matrix:

In [10]:
emission = calculate_emission_from_chroma(chromagram)